In [1]:
import pandas as pd
import plotly_express as px

In [2]:
# Load the HTML table into a DataFrame
trades = pd.read_html('./Statement.htm')
df = trades[0].copy()  

header_row_index = 2
df.columns = df.iloc[header_row_index]

# Remove the header row and rows above it
df = df[header_row_index + 1:]

# Drop unwanted columns without using inplace=True
df = df.drop(['Ticket', 'Taxes'], axis=1)

# Rename columns
df.columns = ['Open Date', 'Type', 'Size', 'Asset', 'Open Price', 'Stop Loss', 'Take Profit', 'Close Date', 'Close Price', 'Commissions', 'Swap', 'PnL']

# Filter rows based on 'Type'
valid_trade_types = ['buy', 'sell', 'buy limit', 'sell limit', 'buy stop', 'sell stop']
df = df[df['Type'].str.lower().isin(valid_trade_types)]

# Filter out rows where 'PnL' equals 'cancelled'
df = df[df['PnL'].str.lower() != 'cancelled']

# Convert specific columns to float
cols_to_float = ['Size', 'Open Price', 'Stop Loss', 'Take Profit', 'Close Price', 'Commissions', 'Swap', 'PnL']
for col in cols_to_float:
    df[col] = df[col].astype(str).str.replace(' ', '').astype(float)

df['Open Date'] = pd.to_datetime(df['Open Date'], format='%Y.%m.%d %H:%M:%S')
df['Close Date'] = pd.to_datetime(df['Close Date'], format='%Y.%m.%d %H:%M:%S')
df = df.sort_values(by='Close Date', ascending=True)
df['Rolling PnL'] = df['PnL'].cumsum()



In [15]:
negative_df = df[df['PnL'] < 0]
positive_df = df[df['PnL'] > 0]

In [22]:
time_differences_negative = negative_df['Close Date'] - negative_df['Open Date']
time_differences_positive = positive_df['Close Date'] - positive_df['Open Date']
average_time_positive = time_differences_positive.mean()


Timedelta('0 days 10:25:05.583333333')